## 🔁 GEPA: 유전 진화 프롬프트 아키텍처

이 노트북은 LLM 피백 루프를 사용하여 시간이 지남에 따라 더 나은 프롬프트를 진화시키는 자동 프롬프트 최적화 프레임워크인 **GEPA**를 구현합니다. **OpenAI GPT 모델**과 **Google Gemini 모델**을 모두 지원하여 유연한 구성과 빠른 실험을 가능하게 합니다.

-----

### 🚀 주요 기능

  * 일련의 훈련 작업에 대해 시드(seed) 프롬프트를 **평가합니다**.
  * 사용자 지정 평가 함수를 사용하여 출력 품질의 **점수를 매깁니다**.
  * 강력한 '리플렉터(reflector)' 모델(예: Gemini 2.5 Pro)을 사용하여 **프롬프트를 변형합니다**.
  * 여러 작업에 걸쳐 파레토 최적화(Pareto optimization)를 기반으로 **새로운 프롬프트를 선택합니다**.
  * 실행 중 성능이 가장 좋은 프롬프트를 **추적하고 표시합니다**.

-----

### 🔧 필요한 설정

실행하기 전에 다음을 확인하세요:

✅ API 키를 저장하세요:



✅ 다음을 설정하세요:

  * `OPENAI_MODEL_NAME` (OpenAI GPT 사용 시) **또는**
  * `TARGET_MODEL_NAME` 및 `REFLECTOR_MODEL_NAME` (Gemini 사용 시)

✅ 다음을 제공하세요:

  * 최적화할 `SEED_PROMPT`.
  * 각각 다음을 포함하는 `TRAINING_DATA` 항목 목록:
      * `input`: 입력 텍스트.
      * `expected_keywords`: 좋은 출력에 포함되어야 할 키워드 목록.

✅ `BUDGET`(모델을 쿼리할 수 있는 횟수)을 정의하세요.

-----

### 🧠 사용 예시

여러분의 시드 프롬프트가 모델에게 텍스트 요약을 요청한다고 가정해 봅시다. GEPA는 다음을 수행합니다:

1.  예시 입력에 대해 해당 프롬프트가 얼마나 잘 수행되는지 점수를 매깁니다.
2.  성능을 기반으로 Gemini에게 프롬프트를 수정하도록 요청합니다.
3.  여러 세대에 걸쳐 새로운 프롬프트를 테스트하고 진화시킵니다.
4.  마지막에 **가장 잘 진화된 프롬프트**를 반환합니다.

-----

### 📦 포함된 구현 사항

  * ✅ `openai` 라이브러리를 통한 `OpenAI API` 지원
  * ✅ `google-generativeai`를 통한 `Google Generative AI` 지원
  * ✅ 반성적 피드백 루프 (Reflective feedback loop)
  * ✅ 파레토 기반 후보 선택
  * ✅ 예시 평가 함수 (키워드 매칭 방식 — 쉽게 맞춤 설정 가능)

-----

### 🧪 추천 대상

  * 프롬프트 엔지니어링 자동화를 연구하는 연구원
  * **LLM API용 프롬프트를 최적화**하려는 팀
  * **피드백을 통한 LLM 자가 개선** 교육용 데모

-----

### 📈 추천 모델

  * OpenAI: `gpt-4o`
  * Google: `gemini-2.5-flash`, `gemini-2.5-pro`

> ⚠️ OpenAI 및 Gemini 모델을 사용하려면 **API 액세스 권한**이 필요합니다.

이 코드는 **GEPA(Genetic-Evolutionary Prompt Architecture)** 라는 프롬프트 최적화 프레임워크를 구현한 것입니다. 쉽게 말해, **AI가 스스로 프롬프트를 테스트하고, 실패로부터 학습하여 더 나은 프롬프트를 만들어내는 '프롬프트 자동 튜닝 시스템'**입니다.

---
### ## 핵심 구성 요소와 역할

이 시스템은 역할이 다른 두 개의 AI 모델을 사용합니다.

1.  **최적화 대상 모델 (Target Model): `gpt-4o-mini`**
    * 이 모델이 바로 우리가 **성능을 향상시키고 싶은 '선수'**입니다.
    * 해커톤의 과제인 '뉴스 기사 분류'를 실제로 수행하는 모델입니다.
    * 우리의 목표는 이 `gpt-4o-mini`가 최고의 성능을 내도록 만드는 최적의 지시서(시스템 프롬프트)를 찾는 것입니다.

2.  **프롬프트 개선 모델 (Reflector Model): `gemini-2.5-pro`**
    * 이 모델은 시스템 전체의 **'두뇌'이자 '코치'** 역할을 합니다.
    * `gpt-4o-mini`가 특정 프롬프트로 작업을 수행했을 때의 성공 또는 실패 결과를 보고, **"왜 실패했을까?"** 또는 **"어떻게 하면 더 잘할 수 있을까?"**를 분석합니다.
    * 그 분석을 바탕으로 기존 프롬프트를 수정하여 더 나은 버전의 새로운 프롬프트를 제안(생성)합니다. 일반적으로 대상 모델보다 더 크고 강력한 모델을 사용하여 깊이 있는 분석이 가능하게 합니다.



---
### ## 최적화 과정 (자동화 루프)

코드는 `BUDGET`(예산)이 소진될 때까지 다음과 같은 과정을 자동으로 반복합니다.

* **1단계: 초기 평가 (Initial Evaluation)**
    1.  먼저, 사람이 작성한 `SEED_PROMPT`(초기 프롬프트)를 `gpt-4o-mini`에게 줍니다.
    2.  CSV 파일에서 읽어온 모든 `TRAINING_DATA`(훈련 데이터)에 대해 이 프롬프트로 작업을 수행시킵니다.
    3.  `evaluation_and_feedback_function`(평가 함수)가 각 작업의 결과를 채점하여 초기 평균 점수를 계산하고, 이 프롬프트를 '후보 1번'으로 저장합니다.

* **2단계: '반성'을 통한 분석 (Reflection)**
    1.  현재까지 만들어진 프롬프트 후보 중 하나를 선택합니다.
    2.  훈련 데이터 중 무작위로 하나를 뽑아 `gpt-4o-mini`에게 다시 작업을 시킵니다.
    3.  평가 함수가 결과를 채점하고, "실패: 예상 값은 '1'이었지만, 결과는 '0'이었습니다."와 같은 간단한 피드백을 만듭니다.
    4.  이 **피드백, 사용된 프롬프트, 입력 데이터, 그리고 모델의 잘못된 출력**을 모두 **`gemini-2.5-pro`(코치)**에게 전달합니다.

* **3단계: 새로운 프롬프트 생성 (Mutation)**
    1.  `gemini-2.5-pro`는 전달받은 정보를 종합적으로 분석하여 기존 프롬프트의 문제점을 해결하고, 길이 점수까지 고려하여 더 간결하고 효율적인 **새로운 프롬프트**를 생성합니다.

* **4단계: 새 후보 평가 및 선택 (Evaluation & Selection)**
    1.  새롭게 생성된 프롬프트를 다시 모든 `TRAINING_DATA`에 대해 테스트하여 평균 점수를 계산합니다.
    2.  이 점수가 현재까지의 최고 점수보다 높거나 같으면, 새로운 '후보'로 채택하여 다음 사이클에 사용될 수 있도록 저장합니다. 점수가 낮으면 폐기합니다.

* **5단계: 반복**
    1.  정해진 `BUDGET` 횟수만큼 2~4단계를 계속 반복합니다.
    2.  루프가 모두 끝나면, 그동안 만들어진 모든 후보 프롬프트 중 **가장 높은 평균 점수를 기록한 최종 프롬프트**를 화면에 출력해 줍니다.

---
### ## 비유하자면...

이 과정은 마치 **'프롬프트 엔지니어링 개인 트레이너'**와 같습니다.

* **선수 (`gpt-4o-mini`)**가 특정 지시서(프롬프트)를 보고 훈련(뉴스 분류)을 합니다.
* **코치 (`gemini-2.5-pro`)**가 선수의 훈련 모습을 비디오로 분석하여 "이번엔 이 부분이 문제였으니, 다음엔 이렇게 해보자"라고 더 좋고 간결한 지시서를 새로 써줍니다.
* 선수는 새로운 지시서로 더 나은 성과를 내고, 이 과정을 반복하여 최고의 기록을 달성하는 것입니다.

In [ ]:
# --- 1. 설치 및 라이브러리 임포트 ---

import os
import random
import time
import openai
from google import genai
from google.genai import types
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

# --- 헬퍼 함수 ---

def log_message(message, type='info'):
    """타임스탬프와 함께 로그 메시지 형식을 지정하는 헬퍼 함수."""
    timestamp = time.strftime("%H:%M:%S")
    if type == 'success':
        return f"[{timestamp}] ✅ 성공: {message}"
    if type == 'fail':
        return f"[{timestamp}] ❌ 실패: {message}"
    if type == 'best':
        return f"[{timestamp}] ⭐ 최고: {message}"
    return f"[{timestamp}] ℹ️ 정보: {message}"

# --- GEPA 핵심 함수 ---

def run_openai_rollout(client: openai.Client, model_id: str, prompt: str, input_text: str) -> str:
    """
    대상 모델(gpt-4o-mini)에 대해 OpenAI API를 호출합니다.
    """
    try:
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": input_text}
        ]
        response = client.chat.completions.create(
            model=model_id,
            messages=messages,
            max_tokens=5,
            temperature=0.4, # 대회 규정에 맞춰 0.4로 고정
            top_p=0.95
        )
        return response.choices[0].message.content.strip()
    except openai.APIError as e:
        err_str = str(e).lower()
        if "authentication" in err_str or "401" in err_str:
            raise Exception(f"OpenAI API 오류: 인증 실패. OpenAI API 키를 확인하세요.")
        if "rate limit" in err_str or "429" in err_str:
            raise Exception(f"OpenAI API 오류: 요청 한도 초과. 잠시 후 다시 시도하세요.")
        if "not found" in err_str or "404" in err_str:
             raise Exception(f"OpenAI API 오류: 모델 '{model_id}'를 찾을 수 없습니다.")
        raise Exception(f"OpenAI API 오류: {str(e)}")
    except Exception as e:
        raise Exception(f"OpenAI 롤아웃 중 알 수 없는 오류 발생: {str(e)}")


def evaluation_and_feedback_function(output, task):
    """
    모델의 출력이 정확히 '0' 또는 '1'인지 채점하고 피드백을 제공합니다.
    """
    if not output or not isinstance(output, str):
        return {"score": 0.0, "feedback": "실패: 유효한 출력이 생성되지 않았습니다."}

    expected_output = task.get("expected_output")

    if output not in ["0", "1"]:
        return {"score": 0.0, "feedback": f"실패: 출력 형식이 잘못되었습니다. (출력: '{output}', 예상: '{expected_output}')"}

    if output == expected_output:
        score = 1.0
        feedback = "성공: 예상된 값과 정확히 일치했습니다."
    else:
        score = 0.0
        feedback = f"실패: 예상 값은 '{expected_output}'이었지만, 결과는 '{output}'이었습니다."

    return {"score": score, "feedback": feedback}


def reflect_and_propose_new_prompt(gemini_client, reflector_model_name, current_prompt, examples):
    """
    강력한 LLM(Gemini)을 사용하여 '반성적 프롬프트 변형' 단계를 수행합니다. (genai.Client 사용)
    """
    examples_text = '---\n'.join(
        f'과제 입력:\n{e["input"]}\n\n생성된 출력: "{e["output"]}"\n피드백: {e["feedback"]}\n\n'
        for e in examples
    )

    reflection_prompt = f"""당신은 전문 프롬프트 엔지니어입니다. 당신의 임무는 이전 시도에서 얻은 피드백을 바탕으로 gpt-4o-mini 모델의 성능을 개선하는 것입니다.

    개선이 필요한 현재 프롬프트는 다음과 같습니다:
    --- 현재 프롬프트 ---
    {current_prompt}
    --------------------

    다음은 이 프롬프트가 몇 가지 과제에서 어떻게 수행되었는지, 그리고 무엇이 잘되었고 잘못되었는지에 대한 피드백 예시입니다:
    --- 예시 및 피드백 ---
    {examples_text}
    -------------------------

    이 분석을 바탕으로, 새롭고 개선된 프롬프트를 작성하는 것이 당신의 임무입니다.
    새로운 프롬프트는 다음 규칙을 반드시 따라야 합니다:
    1. 실패 원인을 직접적으로 해결하고 성공 전략을 반영해야 합니다.
    2. 평가에 길이 점수가 있으므로, 가능한 가장 간결하고 효율적으로 작성되어야 합니다.

    당신의 응답은 오직 새로운 프롬프트 텍스트만 포함해야 하며, 그 외의 내용은 없어야 합니다."""

    try:
        # genai.Client를 사용하여 API 호출
        response = gemini_client.models.generate_content(
            model=reflector_model_name,
            contents=reflection_prompt,
            config=types.GenerateContentConfig(
                temperature=0.2,
                top_p=0.95,
                top_k=30,
                thinking_config=types.ThinkingConfig(thinking_budget=2048)
            )
        )
        return response.text.strip()
    except Exception as e:
        raise Exception(f"Gemini API 오류: {str(e)}. Gemini API 키를 확인하세요.")


def select_candidate_for_mutation(candidate_pool, num_tasks):
    """파레토 기반 전략에 따라 변형할 다음 후보를 선택합니다."""
    if not candidate_pool:
        return None
    if len(candidate_pool) == 1:
        return candidate_pool[0]

    best_scores_per_task = [-1.0] * num_tasks
    for candidate in candidate_pool:
        for i in range(num_tasks):
            if candidate["scores"][i] > best_scores_per_task[i]:
                best_scores_per_task[i] = candidate["scores"][i]

    pareto_front_ids = {
        c["id"]
        for c in candidate_pool
        for i in range(num_tasks)
        if abs(c["scores"][i] - best_scores_per_task[i]) < 1e-6
    }

    if not pareto_front_ids:
        return max(candidate_pool, key=lambda c: c["avg_score"])

    selected_id = random.choice(list(pareto_front_ids))
    return next(c for c in candidate_pool if c["id"] == selected_id)


def test_model_connection(client, model_id):
    """모델에 접근 가능하고 작동하는지 테스트합니다."""
    try:
        response = run_openai_rollout(client, model_id, "Say hello", "Hello, world!")
        return True, response
    except Exception as e:
        return False, str(e)


def run_gepa_optimization(openai_key, gemini_key, model_id, reflector_model_name, seed_prompt, training_data, budget):
    """
    GEPA 최적화 프로세스를 총괄하는 메인 함수.
    """
    # --- 초기화 ---
    print(log_message("GEPA 최적화 프로세스를 시작합니다..."))
    openai_client = openai.OpenAI(api_key=openai_key)
    # google.generativeai.Client 초기화
    gemini_client = genai.Client(api_key=gemini_key)

    rollout_count = 0
    candidate_pool = []
    best_candidate = {"prompt": "초기화 중...", "avg_score": -1.0}

    # --- 모델 연결 테스트 ---
    print(log_message(f"대상 모델 연결 테스트 중: {model_id}"))
    connection_ok, test_result = test_model_connection(openai_client, model_id)
    if not connection_ok:
        print(log_message(f"모델 연결 실패: {test_result}", 'fail'))
        raise Exception(f"모델 '{model_id}'에 연결할 수 없습니다: {test_result}")
    print(log_message("모델 연결 성공!", 'success'))

    # --- 초기 시드 프롬프트 평가 ---
    print("\n" + "="*50)
    print(log_message("1단계: 초기 시드 프롬프트 평가"))
    initial_candidate = {"id": 0, "prompt": seed_prompt, "parentId": None, "scores": [0.0] * len(training_data), "avg_score": 0.0}
    total_score = 0.0
    for i, task in enumerate(training_data):
        print(log_message(f"  - 시드 평가 중 ({i+1}/{len(training_data)} 번째 작업)..."))
        try:
            output = run_openai_rollout(openai_client, model_id, initial_candidate["prompt"], task["input"])
            eval_result = evaluation_and_feedback_function(output, task)
            initial_candidate["scores"][i] = eval_result["score"]
            total_score += eval_result["score"]
        except Exception as e:
            print(log_message(f"작업 {i+1}에서 오류 발생: {str(e)}", 'fail'))
            initial_candidate["scores"][i] = 0.0
        finally:
            rollout_count += 1

    initial_candidate["avg_score"] = total_score / len(training_data) if training_data else 0.0
    candidate_pool.append(initial_candidate)
    best_candidate = initial_candidate

    print(log_message(f"시드 프롬프트 초기 점수: {initial_candidate['avg_score']:.2f}", 'best'))
    print(f"현재 최적의 프롬프트:\n---\n{best_candidate['prompt']}\n---")

    # --- 메인 최적화 루프 ---
    print("\n" + "="*50)
    print(log_message(f"2단계: 최적화 루프 시작 (예산: {budget} 롤아웃)"))
    while rollout_count < budget:
        print(log_message(f"--- 반복 시작 (롤아웃: {rollout_count}/{budget}) ---"))

        parent_candidate = select_candidate_for_mutation(candidate_pool, len(training_data))
        print(log_message(f"후보 #{parent_candidate['id']} (점수: {parent_candidate['avg_score']:.2f})를 변형 대상으로 선택했습니다."))

        task_index = random.randint(0, len(training_data) - 1)
        reflection_task = training_data[task_index]
        print(log_message(f"작업 {task_index + 1}을 사용하여 반성적 변형을 수행합니다..."))

        try:
            rollout_output = run_openai_rollout(openai_client, model_id, parent_candidate["prompt"], reflection_task["input"])
            rollout_count += 1
            eval_result = evaluation_and_feedback_function(rollout_output, reflection_task)

            new_prompt = reflect_and_propose_new_prompt(gemini_client, reflector_model_name, parent_candidate["prompt"], [{
                "input": reflection_task["input"], "output": rollout_output, "feedback": eval_result["feedback"]
            }])

            new_candidate = {"id": len(candidate_pool), "prompt": new_prompt, "parentId": parent_candidate["id"], "scores": [0.0] * len(training_data), "avg_score": 0.0}
            print(log_message(f"새로운 후보 프롬프트 #{new_candidate['id']}를 생성했습니다."))

            new_total_score = 0.0
            for i, task in enumerate(training_data):
                if rollout_count >= budget:
                    print(log_message("새 후보 평가 중 예산을 모두 소진했습니다.", 'fail'))
                    break
                try:
                    output = run_openai_rollout(openai_client, model_id, new_candidate["prompt"], task["input"])
                    eval_result = evaluation_and_feedback_function(output, task)
                    new_candidate["scores"][i] = eval_result["score"]
                    new_total_score += eval_result["score"]
                except Exception as e:
                    print(log_message(f"새 후보를 작업 {i+1}에서 평가 중 오류 발생: {str(e)}", 'fail'))
                    new_candidate["scores"][i] = 0.0
                finally:
                    rollout_count += 1

            new_candidate["avg_score"] = new_total_score / len(training_data) if training_data else 0.0

            if new_candidate["avg_score"] >= best_candidate["avg_score"]:
                print(log_message(f"새 후보 #{new_candidate['id']} 성능 향상 또는 유지! 점수: {new_candidate['avg_score']:.2f} >= {best_candidate['avg_score']:.2f}", 'success'))
                candidate_pool.append(new_candidate)
                if new_candidate["avg_score"] > best_candidate["avg_score"]:
                    best_candidate = new_candidate
                    print(log_message("새로운 최적의 프롬프트를 찾았습니다!", 'best'))
                    print(f"현재 최적의 프롬프트:\n---\n{best_candidate['prompt']}\n---")
            else:
                print(log_message(f"새 후보 #{new_candidate['id']}는 성능이 향상되지 않았습니다. 점수: {new_candidate['avg_score']:.2f}. 폐기합니다.", 'fail'))

        except Exception as e:
            print(log_message(f"최적화 반복 중 오류 발생: {str(e)}", 'fail'))
            rollout_count += 1

    print("\n" + "="*50)
    print(log_message("최적화 예산을 모두 소진했습니다. 종료합니다.", 'best'))
    print(f"최종 최적 프롬프트 (점수: {best_candidate['avg_score']:.2f}):")
    print(f"\n{best_candidate['prompt']}\n")
    print("="*50)
    return best_candidate

# ==============================================================================
#                                메인 실행 블록
# ==============================================================================

if __name__ == '__main__':
    # --- 2. 설정 ---
    # 여기에 최적화 실행을 위한 파라미터를 설정하세요.

    # --- API 키 (Colab Secrets에서 로드) ---
    try:
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')    
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    except Exception as e:
        print("Colab Secrets에서 API 키를 로드할 수 없습니다. 'OPENAI_API_KEY'와 'GEMINI_API_KEY'가 설정되었는지 확인하세요.")
        OPENAI_API_KEY, GEMINI_API_KEY = None, None

    # --- 모델 설정 ---
    MODEL_ID = "gpt-4o-mini"
    REFLECTOR_MODEL_NAME = "gemini-2.5-pro"

    # --- 초기 프롬프트 설정 ---
    SEED_PROMPT = """당신은 뉴스 기사 분류 전문가입니다. 제시된 기사가 자동차 산업과 관련이 있는지 판단하여 숫자로만 응답하세요.

【판단 기준】
자동차 관련(1)으로 분류:
- 자동차 제조사가 주요 주체인 기사
- 자동차/전기차/수소차의 생산, 판매, 개발 내용
- 자동차 전용 부품/기술: 배터리(전기차용), 반도체(차량용), 타이어, 유리, 파워트레인, 열관리시스템, 카메라모듈(차량용), AP모듈(차량용), EVCC
- 자동차 직접 서비스: 충전소, 정비, 자동차보험, 카셰어링
- 자율주행 기술(차량 적용 명시)
- 모빌리티 서비스(UAM 포함)

자동차 무관(0)으로 분류:
- 자동차가 단순 예시로만 언급
- 범용 기술/부품(자동차 특정 언급 없음): 일반 반도체, 일반 배터리, 일반 AI/로봇
- 에너지/ESS(주택용/산업용)
- 무역정책/관세(자동차 특정 언급 없음)
- 타 산업 중심(방산, 항공, 철도 등)

【핵심 식별 패턴】
✓ 기업명+자동차 사업: 현대차, 기아, GM, 테슬라, BYD, 도요타 등
✓ 자동차 전용 키워드: 전기차, EV, 자율주행차, ADAS, SDV, 충전인프라, OEM(자동차)
✓ 부품사+자동차 공급: "차량용", "자동차용", "전기차용", "자동차 OEM 공급"

【경계 사례 판단】
- 배터리 → 전기차용 명시(1) / ESS·주택용(0)
- 반도체 → 차량용·자율주행용(1) / 일반·데이터센터용(0)
- AI/로봇 → 자율주행·차내 시스템(1) / 산업용·일반(0)
- 인프라 → 충전소·자율주행도로(1) / 일반 스마트시티(0)
- M&A/투자 → 자동차 기업·부품사(1) / 타업종(0)

【중요】
- 제목과 본문 전체를 종합 판단
- 자동차가 핵심 주제인지 확인
- 애매한 경우 본문의 주요 논점 기준
- 반드시 0 또는 1만 출력
- 설명이나 이유 없이 숫자만 응답

출력: 0 또는 1"""

    # --- 훈련 데이터 설정 (CSV 파일로부터 로드) ---
    TRAINING_DATA = []
    try:
        # Colab에 업로드된 CSV 파일명을 정확히 입력하세요.
        df = pd.read_csv('./data/car_samples.csv')
        for index, row in df.iterrows():
            # 대회 유저 프롬프트 형식에 맞춰 input 데이터 구성
            input_text = f"[기사]\n\n제목: {row['title']}\n\n내용: {row['content']}"
            TRAINING_DATA.append({
                "input": input_text,
                "expected_output": str(row['label'])
            })
        print(log_message(f"'{df.shape[0]}'개의 훈련 데이터를 CSV 파일로부터 성공적으로 로드했습니다.", 'success'))
    except FileNotFoundError:
        print(log_message("CSV 파일('car_samples.csv')을 찾을 수 없습니다. Colab에 파일을 업로드했는지 확인하세요.", 'fail'))
        TRAINING_DATA = None # 오류 발생 시 실행 중단
    except Exception as e:
        print(log_message(f"CSV 파일 처리 중 오류 발생: {e}", 'fail'))
        TRAINING_DATA = None

    # --- 예산 설정 ---
    BUDGET = 50 # 훈련 데이터가 많아졌으므로 예산을 늘리는 것을 추천합니다.

    # --- 3. 최적화 실행 ---
    if OPENAI_API_KEY and GEMINI_API_KEY and TRAINING_DATA:
      try:
          final_result = run_gepa_optimization(
              openai_key=OPENAI_API_KEY,
              gemini_key=GEMINI_API_KEY,
              model_id=MODEL_ID,
              reflector_model_name=REFLECTOR_MODEL_NAME,
              seed_prompt=SEED_PROMPT,
              training_data=TRAINING_DATA,
              budget=BUDGET
          )
      except Exception as e:
          print(f"\n실행 중 복구할 수 없는 오류가 발생했습니다: {e}")

RuntimeError: asyncio.run() cannot be called from a running event loop